# * Sales Data

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  DIM_CORP_KPI

In [2]:
''' Execute Summary Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summary DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-27, 10:14:17

   -> Summary DataFrame : 1054 rows, 18 columns

TDMDBPR : Disconnected


In [3]:
def my_metric_group(v_name):

    # Get : Parameter
    name = v_name
    result = ''
    
    if re.search('Gross Adds', name): result = 'Gross Adds'
    elif re.search('Inflow M1', name): result = 'Inflow M1'
    elif re.search('Inflow M2', name): result = 'Inflow M2'
    elif re.search('No of AP 1D', name): result = 'No of AP 1D'
    elif re.search('Amt of AP 1D', name): result = 'Amt of AP 1D'
    elif re.search('No of AP In Month', name): result = 'No of AP In Month'
    elif re.search('Amt of AP In Month', name): result = 'Amt of AP In Month'
    # elif re.search('CConnected : onsumer', name): result = 'Gross Adds - Con - Consumer'
    # elif re.search('Connected : DataService', name): result = 'Gross Adds - Con - DataService'
    # elif re.search('Registration : Consumer', name): result = 'Gross Adds - Reg - Consumer'
    # elif re.search('Registration : DataService', name): result = 'Gross Adds - Reg - DataService'
    else: result = 'Unknown' 
    
    return result

In [4]:
''' Add Columns '''

chk_src_df['CHANNEL_GRP'] = np.where(chk_src_df['METRIC_CD'].str.contains('CS$'), 'ALL', 'GEO')
chk_src_df['LOCATION_GRP'] = np.where(chk_src_df['METRIC_NAME'].str.contains('Location'), 'Customer', 'Sales')
chk_src_df['METRIC_GRP'] = chk_src_df.apply(lambda x: my_metric_group(str(x['METRIC_NAME'])), axis=1)
# tmp_df['MAIN_FLAG'] = np.where((tmp_df['METRIC_CD'].str.contains('[0-9]C$|[0-9]H$|[0-9]MCOM$|[0-9]CUS$')) | (tmp_df['METRIC_NAME'].str.contains('New Revenue|Existing Revenue')), 'N', 'Y')
# chk_src_df.tail()

# chk_src_df.loc[chk_src_df['PRODUCT']=='TOL'].tail()

### KPIs Summary

In [5]:
''' PREPAID Summary : 14 rows, 24 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'PREPAID'
# channel_grp = 'GEO'

prepaid_df = chk_src_df.copy()
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==curr_mth]
prepaid_df = prepaid_df.loc[prepaid_df['PRODUCT']==product]
# prepaid_df = prepaid_df.loc[prepaid_df['CHANNEL_GRP']==channel_grp]

prepaid_df['ALL'] = prepaid_df['P']
prepaid_df['TRUE'] = np.where(prepaid_df['COMP_CD']=='TRUE', prepaid_df['P'], 0)
prepaid_df['DTAC'] = np.where(prepaid_df['COMP_CD']=='DTAC', prepaid_df['P'], 0)
prepaid_df['TMP_NAME'] = prepaid_df['METRIC_NAME'].replace(r' : DTAC| : TMH', '', regex=True)

prepaid_df = prepaid_df.groupby(['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
prepaid_df = prepaid_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).reset_index()

mod_col_list = prepaid_df.iloc[:, 6:9].columns.tolist()
for col in mod_col_list:
    prepaid_df[col] = prepaid_df[col].apply(lambda x: format(x, ',.0f'))

prepaid_df

,PRODUCT,METRIC_GRP,TMP_NAME,LOAD_DATE,START_DAY,END_DAY,ALL,TRUE,DTAC
0,PREPAID,Amt of AP 1D,Prepaid Amt of AP 1D,2025-03-27 08:01:09,20250301,20250326,"312,863,231","154,801,743","158,061,488"
1,PREPAID,Amt of AP 1D,Prepaid Amt of AP 1D - GEO Channel,2025-03-27 08:01:09,20250301,20250326,"307,223,505","153,215,688","154,007,817"
2,PREPAID,Amt of AP In Month,Prepaid Amt of AP In Month,2025-03-27 08:01:09,20250301,20250326,"338,940,677","165,004,403","173,936,274"
3,PREPAID,Amt of AP In Month,Prepaid Amt of AP In Month - GEO Channel,2025-03-27 08:01:09,20250301,20250326,"333,111,789","163,397,580","169,714,209"
4,PREPAID,Gross Adds,Prepaid Gross Adds,2025-03-27 08:01:09,20250301,20250326,"1,613,333","790,766","822,567"
5,PREPAID,Gross Adds,Prepaid Gross Adds - GEO Channel,2025-03-27 08:01:09,20250301,20250326,"1,597,792","788,151","809,641"
6,PREPAID,Inflow M1,Prepaid Inflow M1,2025-03-27 08:01:09,20250301,20250325,"332,336,024","153,795,062","178,540,962"
7,PREPAID,Inflow M1,Prepaid Inflow M1 - GEO Channel,2025-03-27 08:01:09,20250301,20250325,"326,935,272","152,332,550","174,602,721"
8,PREPAID,Inflow M2,Prepaid Inflow M2,2025-03-27 08:01:09,20250301,20250325,"101,955,233","48,638,414","53,316,819"
9,PREPAID,Inflow M2,Prepaid Inflow M2 - GEO Channel,2025-03-27 08:01:09,20250301,20250325,"101,339,053","48,567,920","52,771,132"


In [6]:
''' POSTPAID Summary : 8 rows, 16 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'POSTPAID'
# channel_grp = 'GEO'

postpaid_df = chk_src_df.copy()
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==curr_mth]
postpaid_df = postpaid_df.loc[postpaid_df['PRODUCT']==product]
# postpaid_df = postpaid_df.loc[postpaid_df['CHANNEL_GRP']==channel_grp]

postpaid_df['ALL'] = postpaid_df['P']
postpaid_df['TRUE'] = np.where(postpaid_df['COMP_CD']=='TRUE', postpaid_df['P'], 0)
postpaid_df['DTAC'] = np.where(postpaid_df['COMP_CD']=='DTAC', postpaid_df['P'], 0)
postpaid_df['TMP_NAME'] = postpaid_df['METRIC_NAME'].replace(r' : DTAC| : TMH', '', regex=True)

postpaid_df = postpaid_df.groupby(['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
postpaid_df = postpaid_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).reset_index()

mod_col_list = postpaid_df.iloc[:, 6:9].columns.tolist()
for col in mod_col_list:
    postpaid_df[col] = postpaid_df[col].apply(lambda x: format(x, ',.0f'))

postpaid_df

,PRODUCT,METRIC_GRP,TMP_NAME,LOAD_DATE,START_DAY,END_DAY,ALL,TRUE,DTAC
0,POSTPAID,Gross Adds,Postpaid Gross Adds B2B,2025-03-27 08:14:43,20250301,20250326,"33,540","23,151","10,389"
1,POSTPAID,Gross Adds,Postpaid Gross Adds B2B - GEO Channel,2025-03-27 08:14:43,20250301,20250326,"3,254","2,700",554
2,POSTPAID,Gross Adds,Postpaid Gross Adds B2C,2025-03-27 08:14:43,20250301,20250326,"169,125","135,051","34,074"
3,POSTPAID,Gross Adds,Postpaid Gross Adds B2C - GEO Channel,2025-03-27 08:14:43,20250301,20250326,"82,263","70,478","11,785"
4,POSTPAID,Inflow M1,Postpaid Inflow M1 B2B,2025-03-27 08:14:43,20250301,20250326,"13,087,544","4,990,748","8,096,796"
5,POSTPAID,Inflow M1,Postpaid Inflow M1 B2B - GEO Channel,2025-03-27 08:14:43,20250301,20250326,"2,781,319","2,460,785","320,534"
6,POSTPAID,Inflow M1,Postpaid Inflow M1 B2C,2025-03-27 08:14:43,20250301,20250326,"75,377,901","61,108,493","14,269,409"
7,POSTPAID,Inflow M1,Postpaid Inflow M1 B2C - GEO Channel,2025-03-27 08:14:43,20250301,20250326,"40,243,286","33,778,400","6,464,886"


In [7]:
''' TOL Summary : 20 KPIs'''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'TOL'
# channel_grp = 'GEO'

tol_df = chk_src_df.copy()
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==curr_mth]
tol_df = tol_df.loc[tol_df['PRODUCT']==product]
# tol_df = tol_df.loc[tol_df['CHANNEL_GRP']==channel_grp]

tol_df = tol_df.groupby(['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum'})
tol_df = tol_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).reset_index()

mod_col_list = tol_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    tol_df[col] = tol_df[col].apply(lambda x: format(x, ',.0f'))

tol_df

,PRODUCT,METRIC_GRP,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P
0,TOL,Gross Adds,TOL Gross Adds Connected : Consumer,2025-03-27 08:00:09,20250301,20250326,"44,719"
1,TOL,Gross Adds,TOL Gross Adds Connected : Consumer (Install L...,2025-03-27 08:00:09,20250301,20250326,"44,719"
2,TOL,Gross Adds,TOL Gross Adds Connected : Consumer - GEO Channel,2025-03-27 08:00:09,20250301,20250326,"35,202"
3,TOL,Gross Adds,TOL Gross Adds Connected : Consumer - GEO Chan...,2025-03-27 08:00:09,20250301,20250326,"35,202"
4,TOL,Gross Adds,TOL Gross Adds Connected : DataService,2025-03-27 08:00:09,20250301,20250326,"1,395"
5,TOL,Gross Adds,TOL Gross Adds Connected : DataService (Instal...,2025-03-27 08:00:09,20250301,20250326,"1,395"
6,TOL,Gross Adds,TOL Gross Adds Connected : DataService - GEO C...,2025-03-27 08:00:09,20250304,20250326,30
7,TOL,Gross Adds,TOL Gross Adds Connected : DataService - GEO C...,2025-03-27 08:00:09,20250304,20250326,30
8,TOL,Gross Adds,TOL Gross Adds Registration : Consumer,2025-03-27 08:00:09,20250301,20250326,"61,616"
9,TOL,Gross Adds,TOL Gross Adds Registration : Consumer - GEO C...,2025-03-27 08:00:09,20250301,20250326,"47,781"


In [8]:
''' TVS Summary : 8 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'TVS'
# channel_grp = 'GEO'

tvs_df = chk_src_df.copy()
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==curr_mth]
tvs_df = tvs_df.loc[tvs_df['PRODUCT']==product]
# tvs_df = tvs_df.loc[tvs_df['CHANNEL_GRP']==channel_grp]

tvs_df = tvs_df.groupby(['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum'})
tvs_df = tvs_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).reset_index()

mod_col_list = tvs_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    tvs_df[col] = tvs_df[col].apply(lambda x: format(x, ',.0f'))

tvs_df

,PRODUCT,METRIC_GRP,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P
0,TVS,Gross Adds,TVS CMDU Gross Adds,2025-03-27 08:15:58,20250301,20250322,45
1,TVS,Gross Adds,TVS CMDU Gross Adds - GEO Channel,2025-03-27 08:15:58,20250304,20250321,14
2,TVS,Gross Adds,TVS Now Gross Adds,2025-03-27 08:15:58,20250301,20250325,"1,789"
3,TVS,Gross Adds,TVS Now Gross Adds - GEO Channel,2025-03-27 08:15:58,20250301,20250325,"1,787"
4,TVS,Inflow M1,TVS CMDU Inflow M1,2025-03-27 08:15:58,20250301,20250322,"289,613"
5,TVS,Inflow M1,TVS CMDU Inflow M1 - GEO Channel,2025-03-27 08:15:58,20250304,20250321,"53,744"
6,TVS,Inflow M1,TVS Now Inflow M1,2025-03-27 08:15:58,20250301,20250325,"427,778"
7,TVS,Inflow M1,TVS Now Inflow M1 - GEO Channel,2025-03-27 08:15:58,20250301,20250325,"427,555"


### Monitoring

In [9]:
''' Current Month Gross Adds '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['TM_KEY_MTH']==curr_mth]
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-03-27 08:14:43,20250301,20250325,0,"34,074","33,802","33,802","33,802","33,802","33,802",7,5445,32593
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-03-27 08:14:43,20250301,20250325,0,"10,389","3,856","3,856","3,856","3,856","3,856",7,1077,4197
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-03-27 08:14:43,20250301,20250326,0,"135,051","95,868","95,868","95,868","95,868","95,868",7,1441,24516
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-03-27 08:14:43,20250301,20250326,0,"23,151","2,705","2,705","2,705","2,705","2,705",7,713,4407
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-03-27 08:01:09,20250301,20250325,0,"822,567","713,376","713,376","781,219","795,808","791,466",7,6793,86639
5,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,2025-03-27 08:01:09,20250301,20250326,0,"790,766","602,524","602,524","582,112","602,524","602,492",7,5769,94825
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-03-27 08:00:09,20250301,20250326,0,"44,719","35,204","35,204","34,870","35,204","35,204",7,1170,19692
7,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-03-27 08:00:09,20250301,20250326,0,"1,395",30,30,30,30,30,7,68,110
8,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-03-27 08:00:09,20250301,20250326,0,"61,616","47,776","47,776","47,341","47,776","47,776",7,1174,20306
9,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-03-27 08:00:09,20250301,20250326,0,"1,326",55,55,55,55,55,7,86,172


In [10]:
''' Average Gross Adds per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-03-27 08:14:43,20240101,20250228,0,"62,952","61,001","61,001","61,001","61,001","61,001",7,6204,657186
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-03-27 08:14:43,20240101,20250228,0,"9,947","5,395","5,395","5,395","5,395","5,395",7,1528,74902
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-03-27 08:14:43,20240101,20250228,0,"152,413","102,608","102,608","102,608","102,608","102,608",7,2211,409848
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-03-27 08:14:43,20240101,20250228,0,"24,861","8,465","8,465","8,465","8,465","8,465",7,781,59643
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-03-27 08:01:09,20240101,20250228,0,"1,370,010","1,315,762","1,315,762","1,305,571","1,323,604","1,315,515",7,7441,1577122
5,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,2025-03-27 08:01:09,20240101,20250228,0,"1,149,759","954,285","954,285","932,410","954,285","954,219",7,5817,1671756
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-03-27 08:00:09,20240102,20250228,0,"46,522","32,998","32,998","34,286","34,612","34,597",7,1214,307979
7,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-03-27 08:00:09,20240102,20250228,0,"1,574",34,34,35,35,35,7,99,2005
8,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-03-27 08:00:09,20240101,20250228,0,"63,752","44,821","44,821","46,597","47,021","46,996",7,1243,324801
9,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-03-27 08:00:09,20240101,20250228,0,"3,488","1,103","1,103","1,094","1,114","1,114",7,620,25550


In [11]:
''' Current Month Inflow '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['TM_KEY_MTH']==curr_mth]
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-03-27 08:14:43,20250301,20250325,0,"14,269,409","14,116,725","14,116,725","14,116,725","14,116,725","14,116,725",7,5445,32593
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-03-27 08:14:43,20250301,20250325,0,"8,096,796","1,431,357","1,431,357","1,431,357","1,431,357","1,431,357",7,1077,4197
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-03-27 08:14:43,20250301,20250326,0,"61,108,493","48,599,720","48,599,720","48,599,720","48,599,720","48,599,720",7,1441,24516
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-03-27 08:14:43,20250301,20250326,0,"4,990,748","2,463,680","2,463,680","2,463,680","2,463,680","2,463,680",7,713,4407
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-03-27 08:01:09,20250301,20250325,0,"178,540,962","158,009,123","158,009,123","171,514,022","175,033,575","174,053,650",7,6783,126180
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-03-27 08:01:09,20250301,20250325,0,"53,316,819","45,408,957","45,408,957","51,970,848","52,713,482","52,151,678",7,6812,151722
6,PREPAID,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-03-27 08:01:09,20250301,20250324,0,"153,795,062","119,436,427","119,436,427","115,293,228","119,436,427","119,431,215",7,5710,108318
7,PREPAID,TB1R001000CS,Prepaid Inflow M2 : TMH,2025-03-27 08:01:09,20250301,20250324,0,"48,638,414","47,758,516","47,758,516","46,833,571","47,758,516","47,752,675",7,5669,118689
8,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-03-27 08:00:09,20250301,20250326,0,"24,478,283","19,602,973","19,602,973","19,404,368","19,602,973","19,602,973",7,1170,19692
9,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-03-27 08:00:09,20250301,20250326,0,"1,743,453","77,472","77,472","77,472","77,472","77,472",7,68,110


In [12]:
''' Average Inflow per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-03-27 08:14:43,20240101,20250228,0,"381,577,081","372,746,838","372,746,838","372,746,838","372,746,838","372,746,838",7,6204,657186
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-03-27 08:14:43,20240101,20250228,0,"137,554,957","25,803,178","25,803,178","25,803,178","25,803,178","25,803,178",7,1528,74902
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-03-27 08:14:43,20240101,20250228,0,"892,798,861","686,190,728","686,190,728","686,190,728","686,190,728","686,190,728",7,2211,409848
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-03-27 08:14:43,20240101,20250228,0,"78,786,203","33,347,478","33,347,478","33,347,478","33,347,478","33,347,478",7,781,59643
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-03-27 08:01:09,20240101,20250228,0,"3,575,366,338","3,494,329,040","3,494,329,040","3,462,849,849","3,517,392,428","3,493,806,293",7,7418,2373492
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-03-27 08:01:09,20240201,20250228,0,"1,018,625,276","995,812,331","995,812,331","993,869,298","1,005,487,512","993,973,720",7,7294,2559496
6,PREPAID,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-03-27 08:01:09,20240101,20250228,0,"2,736,943,469","2,376,984,888","2,376,984,888","2,308,755,337","2,376,984,888","2,376,829,121",7,5793,1952685
7,PREPAID,TB1R001000CS,Prepaid Inflow M2 : TMH,2025-03-27 08:01:09,20240101,20250228,0,"997,560,166","988,072,190","988,072,190","970,164,151","988,072,190","987,964,726",7,5739,2116490
8,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-03-27 08:00:09,20240102,20250228,0,"359,959,644","260,255,836","260,255,836","270,547,034","273,372,494","273,249,869",7,1214,307979
9,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-03-27 08:00:09,20240102,20250228,0,"26,851,654","1,478,429","1,478,429","1,514,815","1,517,914","1,517,914",7,99,2005


### All Metric

In [13]:
''' METRIC Summary '''

# src_t_sales_mth_df = chk_src_df.copy()
# src_t_sales_mth_df = src_t_sales_mth_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
# src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

# mod_col_list = src_t_sales_mth_df.iloc[:, 6:7].columns.tolist()
# for col in mod_col_list:
#     src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_sales_mth_df.to_string(max_cols=10)}')

' METRIC Summary '

In [14]:
''' METRIC Current Month '''

# # v_tm_key_mth = '202502'
# curr_mth = chk_src_df['TM_KEY_MTH'].max()
# src_t_sales_mth_df_cols = ['PRODUCT', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
# print(f'\nParameter input...\n')
# print(f'   -> v_tm_key_mth: {curr_mth}')

# src_t_sales_mth_df = chk_src_df[src_t_sales_mth_df_cols].copy()
# src_t_sales_mth_df = src_t_sales_mth_df.loc[src_t_sales_mth_df['TM_KEY_MTH']==curr_mth]
# src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

# mod_col_list = src_t_sales_mth_df.iloc[:, 7:14].columns.tolist()
# for col in mod_col_list:
#     src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_sales_mth_df.to_string(max_cols=17)}')

' METRIC Current Month '

## Actual : Transaction
->  DIM_CORP_KPI

In [15]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101

TDMDBPR : Connected

   -> Execute query... 2025-03-27, 10:17:06



   -> Daily DataFrame : 5758 rows, 12 columns

TDMDBPR : Disconnected


In [16]:
''' METRIC by Day '''

v_tm_key_day = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

src_sale_day_df = chk_src_df.copy()
src_sale_day_df = src_sale_day_df.loc[src_sale_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_sale_day_df = src_sale_day_df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_sale_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_sale_day_df[col] = src_sale_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_sale_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_sale_day_df.to_string(max_cols=12)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_sale_day_df...

   -> DataFrame : 5758 rows, 12 columns


In [17]:
''' xB1S000101CS : Prepaid Gross Adds : xxx '''

v_tm_key_day = 20250301
v_metric_cd = ['DB1S000101CS', 'TB1S000101CS']
print(f'\nDB1S000101CS : Prepaid Gross Adds : DTAC\nTB1S000101CS : Prepaid Gross Adds : TMH\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

xB1S000101CS_day_df = chk_src_df.copy()
xB1S000101CS_day_df = xB1S000101CS_day_df.loc[xB1S000101CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
xB1S000101CS_day_df = xB1S000101CS_day_df.loc[xB1S000101CS_day_df['METRIC_CD'].isin(v_metric_cd)]
xB1S000101CS_day_df = xB1S000101CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = xB1S000101CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    xB1S000101CS_day_df[col] = xB1S000101CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
xB1S000101CS_day_df


DB1S000101CS : Prepaid Gross Adds : DTAC
TB1S000101CS : Prepaid Gross Adds : TMH

   -> v_metric_cd: ['DB1S000101CS', 'TB1S000101CS']
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250301,2025-03-27 08:01:09,0,"39,049","33,586","33,586","36,734","37,288","37,073"
1,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250302,2025-03-27 08:01:09,0,"41,058","33,972","33,972","37,310","37,911","37,496"
2,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250303,2025-03-27 08:01:09,0,"35,155","30,338","30,338","33,573","34,442","34,283"
3,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250304,2025-03-27 08:01:09,0,"33,235","28,889","28,889","32,039","32,829","32,692"
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250305,2025-03-27 08:01:09,0,"34,208","29,931","29,931","33,070","33,765","33,632"
5,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250306,2025-03-27 08:01:09,0,"36,486","32,198","32,198","35,114","36,019","35,856"
6,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250307,2025-03-27 08:01:09,0,"38,198","32,973","32,973","35,636","36,527","36,388"
7,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250308,2025-03-27 08:01:09,0,"33,204","29,014","29,014","31,950","32,551","32,396"
8,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250309,2025-03-27 08:01:09,0,"33,337","29,011","29,011","32,019","32,537","32,180"
9,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250310,2025-03-27 08:01:09,0,"32,484","28,389","28,389","31,176","31,792","31,654"


In [18]:
''' xB2S010100CS : Postpaid Gross Adds B2C : xxx '''

v_tm_key_day = 20250301
v_metric_cd = ['DB2S010100CS', 'TB2S010100CS']
print(f'\nDB2S010100CS : Postpaid Gross Adds : DTAC\nTB2S010100CS : Postpaid Gross Adds : TMH\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

xB2S010100CS_day_df = chk_src_df.copy()
xB2S010100CS_day_df = xB2S010100CS_day_df.loc[xB2S010100CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
xB2S010100CS_day_df = xB2S010100CS_day_df.loc[xB2S010100CS_day_df['METRIC_CD'].isin(v_metric_cd)]
xB2S010100CS_day_df = xB2S010100CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = xB2S010100CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    xB2S010100CS_day_df[col] = xB2S010100CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
xB2S010100CS_day_df


DB2S010100CS : Postpaid Gross Adds : DTAC
TB2S010100CS : Postpaid Gross Adds : TMH

   -> v_metric_cd: ['DB2S010100CS', 'TB2S010100CS']
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250301,2025-03-27 08:14:43,0,"1,143","1,138","1,138","1,138","1,138","1,138"
1,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250302,2025-03-27 08:14:43,0,888,887,887,887,887,887
2,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250303,2025-03-27 08:14:43,0,"1,362","1,354","1,354","1,354","1,354","1,354"
3,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250304,2025-03-27 08:14:43,0,"1,696","1,684","1,684","1,684","1,684","1,684"
4,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250305,2025-03-27 08:14:43,0,"1,383","1,373","1,373","1,373","1,373","1,373"
5,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250306,2025-03-27 08:14:43,0,"1,707","1,688","1,688","1,688","1,688","1,688"
6,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250307,2025-03-27 08:14:43,0,"1,366","1,359","1,359","1,359","1,359","1,359"
7,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250308,2025-03-27 08:14:43,0,"1,164","1,156","1,156","1,156","1,156","1,156"
8,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250309,2025-03-27 08:14:43,0,834,832,832,832,832,832
9,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250310,2025-03-27 08:14:43,0,"1,525","1,514","1,514","1,514","1,514","1,514"


In [19]:
''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

v_tm_key_day = 20250301
v_metric_cd = 'TB3S000102CS'
print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3S000102CS_day_df = chk_src_df.copy()
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['METRIC_CD']==v_metric_cd]
TB3S000102CS_day_df = TB3S000102CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3S000102CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3S000102CS_day_df[col] = TB3S000102CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3S000102CS_day_df


TB3S000102CS : TOL Gross Adds Connected : Consumer

   -> v_metric_cd: TB3S000102CS
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250301,2025-03-27 08:00:09,0,"1,759","1,376","1,376","1,365","1,376","1,376"
1,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250302,2025-03-27 08:00:09,0,"1,603","1,211","1,211","1,193","1,211","1,211"
2,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250303,2025-03-27 08:00:09,0,"1,737","1,392","1,392","1,373","1,392","1,392"
3,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250304,2025-03-27 08:00:09,0,"1,789","1,449","1,449","1,441","1,449","1,449"
4,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250305,2025-03-27 08:00:09,0,"1,733","1,417","1,417","1,402","1,417","1,417"
5,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250306,2025-03-27 08:00:09,0,"1,726","1,408","1,408","1,400","1,408","1,408"
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250307,2025-03-27 08:00:09,0,"1,657","1,348","1,348","1,338","1,348","1,348"
7,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250308,2025-03-27 08:00:09,0,"1,806","1,334","1,334","1,323","1,334","1,334"
8,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250309,2025-03-27 08:00:09,0,"1,711","1,265","1,265","1,259","1,265","1,265"
9,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250310,2025-03-27 08:00:09,0,"1,826","1,418","1,418","1,410","1,418","1,418"


In [20]:
''' TB4S001400CS : TVS Now Gross Adds '''

v_tm_key_day = 20250301
v_metric_cd = 'TB4S001400CS'
print(f'\nTB4S001400CS : TVS Now Gross Adds\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4S001400CS_day_df = chk_src_df.copy()
TB4S001400CS_day_df = TB4S001400CS_day_df.loc[TB4S001400CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4S001400CS_day_df = TB4S001400CS_day_df.loc[TB4S001400CS_day_df['METRIC_CD']==v_metric_cd]
TB4S001400CS_day_df = TB4S001400CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4S001400CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4S001400CS_day_df[col] = TB4S001400CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4S001400CS_day_df


TB4S001400CS : TVS Now Gross Adds

   -> v_metric_cd: TB4S001400CS
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TVS,TB4S001400CS,TVS Now Gross Adds,20250301,2025-03-27 08:15:58,0,24,24,24,24,24,24
1,TVS,TB4S001400CS,TVS Now Gross Adds,20250302,2025-03-27 08:15:58,0,44,44,44,44,44,44
2,TVS,TB4S001400CS,TVS Now Gross Adds,20250303,2025-03-27 08:15:58,0,21,21,21,21,21,21
3,TVS,TB4S001400CS,TVS Now Gross Adds,20250304,2025-03-27 08:15:58,0,22,22,22,22,22,22
4,TVS,TB4S001400CS,TVS Now Gross Adds,20250305,2025-03-27 08:15:58,0,78,78,78,78,78,78
5,TVS,TB4S001400CS,TVS Now Gross Adds,20250306,2025-03-27 08:15:58,0,214,214,214,214,214,214
6,TVS,TB4S001400CS,TVS Now Gross Adds,20250307,2025-03-27 08:15:58,0,160,160,160,160,160,160
7,TVS,TB4S001400CS,TVS Now Gross Adds,20250308,2025-03-27 08:15:58,0,141,141,141,141,141,141
8,TVS,TB4S001400CS,TVS Now Gross Adds,20250309,2025-03-27 08:15:58,0,144,144,144,144,144,144
9,TVS,TB4S001400CS,TVS Now Gross Adds,20250310,2025-03-27 08:15:58,0,87,87,87,87,87,87


## Actual : Issue
->  DIM_CORP_KPI

In [21]:
# ''' Execute Issue by case '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# # v_start_date = 20250101
# print(f'\nParameter input...\n')
# # print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ T.TM_KEY_DAY AS TM_KEY_MTH
#         , T.METRIC_CD, T.METRIC_NAME, T.AREA_TYPE, T.AREA_CD, T.AREA_DESC
#         , T.METRIC_VALUE AS TARGET
#         , A.ACTUAL
#         , O.ORGID_P, O.ZONE_TYPE, O.ORGID_G, O.TDS_SGMD, O.ORGID_H, O.HOP_HINT, O.ORGID_HH, O.D_CLUSTER
        
#     FROM CDSAPPO.DIM_CORP_TARGET T

#     LEFT JOIN (
#         SELECT SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
#             , PRODUCT, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC
#             , SUM(METRIC_VALUE) AS ACTUAL, MAX(LOAD_DATE) LOAD_DATE
#         FROM CDSAPPO.DIM_CORP_KPI
#         WHERE METRIC_CD = 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
#         AND AREA_TYPE = 'CCAA'
#         AND TM_KEY_DAY >= 20250101
#         GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC
#     ) A
#         ON A.METRIC_CD = T.METRIC_CD
#         AND A.AREA_CD = T.AREA_CD
#         AND A.TM_KEY_MTH = T.TM_KEY_DAY
        
#     LEFT JOIN (
#         SELECT DISTINCT ORGID_P, ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_H, HOP_HINT, ORGID_HH, D_CLUSTER
#             , SUBSTR(CCAATT,1,2) AS PROVINCE_CD, PROVINCE_ENG 
#             , SUBSTR(CCAATT,1,4) AS CCAA, DISTRICT_EN
#         FROM CDSAPPO.DIM_MOOC_AREA NOLOCK
#         WHERE TEAM_CODE <> 'ไม่ระบุ' AND REMARK <> 'Dummy'
#     ) O
#         ON O.CCAA = T.AREA_CD

#     WHERE T.METRIC_CD = 'TB3S000102CS'
#     AND T.AREA_TYPE = 'CCAA'
#     AND T.TM_KEY_DAY >= 202501

#     --ORDER BY T.TM_KEY_DAY, T.METRIC_CD, T.AREA_TYPE, O.ZONE_TYPE, O.ORGID_G, O.ORGID_H, O.ORGID_HH
# """)


# try:
#     execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
#     print(f'\n   -> Execute query... {execute_datetime}')

#     # Get : Summary DataFrame
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
#     print(f'\n   -> Issue DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')

In [22]:
# ''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

# v_tm_key_mth = 202501
# # v_metric_cd = 'TB3S000102CS'
# print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer\n')
# # print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_mth: {v_tm_key_mth}')

# TB3S000102CS_issue_df = chk_src_df.copy()
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['TM_KEY_MTH']==v_tm_key_mth]
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['ACTUAL'] > 0]
# # TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['ACTUAL'].isna()]
# # TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['METRIC_CD']==v_metric_cd]
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)

# mod_col_list = TB3S000102CS_issue_df.iloc[:, 6:8].columns.tolist()
# for col in mod_col_list:
#     TB3S000102CS_issue_df[col] = TB3S000102CS_issue_df[col].apply(lambda x: format(x, ',.0f'))
    
# # TB3S000102CS_issue_df
# print(f'\n{TB3S000102CS_issue_df.to_string(max_cols=100)}')

## Output File
->  DIM_CORP_KPI

In [23]:
# ''' Execute Sales Monthly '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20240101
# print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ 
#         SUBSTR(TM_KEY_DAY,1,4) AS TM_KEY_YR, SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
#         , CASE WHEN REGEXP_LIKE(METRIC_CD, 'CS$|CG$') THEN 'Y2025' ELSE 'Y2024' END VER_FLAG
#         , PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME, REMARK, UPD_TM, PPN_TM
#         , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P_ACTUAL
#         , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
#         , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
#         , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
#         , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
#         , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
#         , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) "016"
#         , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) "040"
#         , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) "080"
#         , MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY, MAX(LOAD_DATE) LOAD_DATE
#     FROM CDSAPPO.DIM_CORP_KPI
#     WHERE METRIC_CD NOT LIKE 'Error%'
#     AND EXISTS (
#         SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
#         FROM (
#             SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#         ) TMP
#         WHERE TMP_CD = AREA_CD )
#     AND TM_KEY_DAY >= {v_start_date}
#     GROUP BY SUBSTR(TM_KEY_DAY,1,4), SUBSTR(TM_KEY_DAY,1,6), PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME, REMARK, UPD_TM, PPN_TM
#     --ORDER BY 2, 3, PRODUCT, COMP_CD, METRIC_CD 
# """)


# try:
#     execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
#     print(f'\n   -> Execute query... {execute_datetime}')

#     # Get : Sales Monthly DataFrame
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     monthly_sales_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
#     print(f'\n   -> Sales Monthly DataFrame : {monthly_sales_df.shape[0]} rows, {monthly_sales_df.shape[1]} columns')

#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')

In [24]:
# ''' Generate CSV file '''

# op_dir = 'data'
# op_monthly_file = f'sales_monthly_{str_curr_dt}'

# monthly_sales_df.to_csv(f'{op_dir}/tmp_{op_monthly_file}.csv', index=False, encoding='utf-8')
# print(f'\n   -> Generate "tmp_{op_monthly_file}.csv" successfully')

# monthly_sales_df.tail(3)